In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201214'
endDate = '20210108'

readPath = '/mnt/orderLog/data/***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)


for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
try:
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r1, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])

try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders out of 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')
df = orderLog.copy()


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
taking = orderLog[orderLog['ars'].isin([1, 7])]
making = orderLog[orderLog['ars'].isin([2, 3])]
else_orders = orderLog[~(orderLog['ars'].isin([1, 2, 3, 7]))]
display('orders with abnormal ars values')
display(else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False))
display(else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False))
# else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False).reset_index()['ars'].values
# kk = else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False)
# kk[kk > 20].index.values
taking['directNum'] = taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].transform('nunique')

if len(taking[taking['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    taking = taking[taking['directNum'] == 1]

assert((taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].nunique() == 1).all() == True)
orderLog = pd.concat([taking, making]).sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())      
    d_el = pd.merge(orderLog, a, on=['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(d_el))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))
display(checkLog[checkLog['status'] == 3].groupby('updateType')['order'].size())
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
try:
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
except:
    print(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
try:
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
except:
    print(orderLog[orderLog['mrss90'] == '-'])
    orderLog = orderLog[orderLog['mrss90'] != '-']
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
try:
    orderLog['aaa'] = orderLog['aaa'].astype(float)
except:
    print(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['/mnt/orderLog/data/20201214.pkl' '/mnt/orderLog/data/20201215.pkl'
 '/mnt/orderLog/data/20201216.pkl' '/mnt/orderLog/data/20201217.pkl'
 '/mnt/orderLog/data/20201218.pkl' '/mnt/orderLog/data/20201221.pkl'
 '/mnt/orderLog/data/20201222.pkl' '/mnt/orderLog/data/20201223.pkl'
 '/mnt/orderLog/data/20201224.pkl' '/mnt/orderLog/data/20201225.pkl'
 '/mnt/orderLog/data/20201228.pkl' '/mnt/orderLog/data/20201231.pkl'
 '/mnt/orderLog/data/20201229.pkl' '/mnt/orderLog/data/20201230.pkl'
 '/mnt/orderLog/data/20210104.pkl' '/mnt/orderLog/data/20210105.pkl'
 '/mnt/orderLog/data/20210106.pkl' '/mnt/orderLog/data/20210107.pkl'
 '/mnt/orderLog/data/20210108.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
106636,20201214,zt_96_09,965801,1601222,-1,7,-1.0,0,0,-1.0,8.240000e+02
295137,20201214,zs_94_03,9461,2002044,-1,7,34444.0,0,0,-1.0,-1.000000e+00
330239,20201214,zs_64_01,6480,2002293,-1,1,47893.0,0,0,-1.0,1.038220e+05
335644,20201214,zs_52_06,5269,2002329,-1,1,37688.0,0,0,-1.0,6.962440e+05
342409,20201214,zs_52_09,5291,2002390,-1,1,51351.0,0,0,-1.0,1.331409e+06
346335,20201214,zs_64_01,6480,2002411,-1,1,53411.0,0,0,-1.0,1.516870e+05
354862,20201214,zs_54_01,5474,2002472,-1,1,47572.0,0,0,-1.0,8.689118e+08
422367,20201214,zs_52_06,5269,2002845,-1,1,39762.0,0,0,-1.0,9.018290e+05
431413,20201214,zs_52_06,5269,2002890,-1,1,37463.0,0,0,-1.0,6.683230e+05
539711,20201214,zs_52_06,5269,2300410,-1,1,35229.0,0,0,-1.0,3.052970e+05


There are orders with all things same except sdd
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
1726162   3686264.0   0.00044                    0           100   
1726443  18252604.0  0.001406                    0           100   
1758144   3688220.0  0.000419                    0           100   
1758167  14404537.0  0.000657                    0           100   
8990406   1444460.0  0.000687                    0           100   
8990466   9795909.0  0.000694                    0           100   

         absOrderSizeCumFilled  accCode  ars             caamd  \
1726162                      0   966301  0.0                 0   
1726443                      0   966301  0.0                 0   
1758144                      0   966301  0.0                 0   
1758167                      0   966301  0.0                 0   
8990406                      0   966301  1.0  1609205575014635   
8990466                      0   966301  1.0  1609208372125359   

         ca

There are orders with same internalId but different orderId other than accCode 8856 case
date      colo      accCode  secid    orderDirection  absOrderSize  internalId
20201216  zt_96_01  966301   2002572   1              100            104.0        2
                             2002745   1              100            55.0         2
20201225  zs_52_09  5291     2002212   0              0             -1.0          2
20201228  zs_54_01  5456     2300036  -1              1000           2093.0       2
20201229  zt_96_01  966301   1600376   0              0             -1.0          3
                             2002595  -1              100            9.0          2
Name: orderId, dtype: int64


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,resa,Unnamed: 0,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status
9882637,0.0,-1,0,100,0,8854,1.0,1609307843130063,1.0,5423341.79,2020-12-30 13:57:23.309056,1609307843309056,0.0,0.0,100.0,0.0,20201230,0.0,5423340.79,-1.0,0.0,0.0,190.0,100.0,0.0,0.0,0.0,1345.000000|,0.0,0.0,0.0,-1.0,1957.0,1951.0,-0.004078,0.000603,-1.0,-1.0,-0.003175,-0.001534,0.000110,-0.001057,13000.0,11000.0,-1.0,17761035.0,13:57:23.307447,22,-1.0,-1,0.0,-1,8.885000e+03,1.0,19.58,5.10089e+07,135722000.0,1603008,10463942.0,2,122924.0,571.0,42.0,NaN,-1.0,852,1,17760235.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.823186,19.564615,19.568571,19.830833,6527.2886,6527.3770,6527.7307,6532.0858,2.0,148575.0,2470376,88,zt_88,2473285,1421054,1609307843130266,178790,-1,1.0,0.0,2
9882749,0.0,-1,0,100,0,8854,1.0,1609298373762077,1.0,4339395.87,2020-12-30 11:19:34.180024,1609298374180024,0.0,0.0,200.0,100.0,20201230,0.0,4339394.87,-1.0,0.0,0.0,287.0,6700.0,0.0,0.0,0.0,1115.000000|,0.0,0.0,0.0,-1.0,3494.0,3475.0,-0.000817,0.002538,-1.0,-1.0,-0.002911,-0.00311,-0.002534,-0.002361,13000.0,11000.0,-1.0,3483963.0,11:19:34.178594,22,-1.0,-1,0.0,-1,6.477000e+03,1.0,34.96,5.10065e+07,111933000.0,1603456,4386801.0,3,114097.0,307.0,20.0,NaN,-1.0,852,1,3483663.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.138497,34.924545,35.016154,34.854000,6534.0868,6535.3699,6538.0923,6536.0611,2.0,189358.0,2470413,88,zt_88,2473320,1426956,1609298373762174,417850,-1,1.0,0.0,2
9882780,0.0,-1,0,200,0,8854,1.0,1609296809192182,1.0,4226087.47,2020-12-30 10:53:29.509160,1609296809509160,600.0,800.0,0.0,0.0,20201230,0.0,4226086.47,-1.0,0.0,0.0,201.0,600.0,600.0,0.0,0.0,1045.000000|,0.0,0.0,0.0,-1.0,7262.0,7261.0,0.001725,0.004244,-1.0,-1.0,0.001422,0.000137,-0.001558,-0.000264,13000.0,11000.0,-1.0,1703499.0,10:53:29.507618,22,-1.0,-1,0.0,1,5.459000e+03,1.0,72.54,5.10055e+07,105328000.0,1603583,2334346.0,3,114104.0,213.0,12.0,NaN,-1.0,852,1,1703299.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583953,72.606667,72.725000,73.204615,6526.7336,6528.0027,6531.0701,6533.7960,2.0,195686.0,2470422,88,zt_88,2473328,1427886,1609296809192395,316765,1,1.0,0.0,2
9882851,0.0,-1,0,100,0,8854,1.0,1609306975678291,1.0,5409570.09,2020-12-30 13:42:56.338860,1609306976338860,0.0,0.0,100.0,0.0,20201230,0.0,5409569.09,-1.0,0.0,0.0,152.0,4250.0,0.0,0.0,0.0,1330.000000|,0.0,0.0,0.0,-1.0,6699.0,6637.0,0.002238,-6.9e-05,-1.0,-1.0,-0.004424,-0.006091,-0.004853,-0.002701,13000.0,11000.0,-1.0,6335649.0,13:42:56.337460,22,-1.0,-1,0.0,-1,8.476000e+03,1.0,67.00,5.10085e+07,134254000.0,1603678,9310481.0,2,122930.0,530.0,39.0,NaN,-1.0,852,1,6317849.0,logs_20201230_zt_88_06_day_8854,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.003676,66.483333,66.538750,66.908438,6523.0712,6522.6492,6523.4124,6526.4314,2.0,205040.0,2470440,88,zt_88,2473349,1429329,1609306975678489,660371,-1,1.0,0.0,2
9882922,0.0,-1,0,100,0,8854,1.0,1609307131162699,1.0,5418525.78,2020-12-30 13:45:31.933621,1609307131933621,2800.0,5600.0,0.0,0.0,20201230,0.0,5418524.78,-1.0,0.0,0.0,170.0,5710.0,5700.0,0.0,0.0,134

11. Orders with size > 80w or notional > 800w
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
5677501         NaN  0.001681                    0           100   

         absOrderSizeCumFilled  accCode  ars             caamd  \
5677501                      0     8967  1.0  1608706221466743   

         cancellationPending          cfe                      clock  \
5677501                  0.0  10375101.08 2020-12-23 14:50:21.466808   

           clockAtArrival  cumSharesBought  cumSharesBuyInserted  \
5677501  1608706221466808              NaN                   NaN   

         cumSharesSellInserted  cumSharesSold      date  finalState  \
5677501                    NaN            NaN  20201223         0.0   

                 gfe       hee  insertedShortOrder  insertionPending  \
5677501  10375101.08  0.001508                 0.0               1.0   

         internalId    inv_L   inv_L0  inv_S  inv_S0 l4algoDebug  l4tr  \
5677501      1593.0  10700.0  10700.0   

,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
35089,20201214,1600346,zt_96_09,966701,0,0.0,29700,1,NaN,NaN,145647000.0,NaN,else
186792,20201214,1603609,zt_96_09,965801,0,0.0,800,1,NaN,NaN,145555000.0,NaN,else
189248,20201214,1603628,zt_52_10,523101,0,0.0,400,-1,NaN,NaN,145642000.0,NaN,else
489934,20201214,2300175,zs_52_06,5222,0,0.0,1100,1,NaN,NaN,145647870.0,NaN,else
694101,20201215,1600248,zt_52_05,522901,0,1040100.0,5400,-1,2000.0,1000.0,93547000.0,0.0,else
694102,20201215,1600248,zt_52_05,522901,2,1040100.0,5400,-1,2000.0,1000.0,93547000.0,0.0,else
694103,20201215,1600248,zt_52_05,522901,4,1040100.0,5400,-1,2000.0,1000.0,93547000.0,0.0,else
694107,20201215,1600248,zt_52_05,522901,0,1183300.0,4500,-1,2000.0,1000.0,93602000.0,0.0,else
694108,20201215,1600248,zt_52_05,522901,2,1183300.0,4500,-1,2000.0,1000.0,93602000.0,0.0,else
694109,20201215,1600248,zt_52_05,522901,4,1183300.0,4500,-1,2000.0,1000.0,93602000.0,0.0,else


mrstaat  mrstauc
1000.0   0.0         524647
3000.0   1000.0     1924257
         2000.0      140562
         3000.0       54915
11000.0  10000.0      41325
         20000.0       3396
         30000.0       2213
13000.0  11000.0     986010
         12000.0      31763
         13000.0       6191
         21000.0      28103
         22000.0      29485
         23000.0       5449
         31000.0      13123
         32000.0      10580
         33000.0      12658
Name: date, dtype: int64

In [2]:
print('SZE')
totalAccount = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['colo'].str[:5] == 'zs_52')]['accCode'].unique()
for date, dateLog in orderLog[(orderLog['secid'] >= 2000000) & (orderLog['colo'].str[:5] == 'zs_52')].groupby(['date']):
    dailyAccount = dateLog['accCode'].unique()
    print(date, sorted(list(set(totalAccount) - set(dailyAccount))))
print(totalAccount)
print('SSE')
totalAccount = orderLog[(orderLog['secid'] < 2000000) & (orderLog['colo'].str[:5] == 'zt_52')]['accCode'].unique()
for date, dateLog in orderLog[(orderLog['secid'] < 2000000) & (orderLog['colo'].str[:5] == 'zt_52')].groupby(['date']):
    dailyAccount = dateLog['accCode'].unique()
    print(date, sorted(list(set(totalAccount) - set(dailyAccount))))
print(totalAccount)

SZE
20201214 [5326, 5329]
20201215 [5326, 5329]
20201216 [5326, 5329]
20201217 [5326, 5329]
20201218 [5326, 5329]
20201221 [5326, 5329]
20201222 [5226, 5329]
20201223 [5226, 5329]
20201224 [5226, 5329]
20201225 [5226, 5229, 5377]
20201228 [5226, 5229, 5377]
20201229 [5226, 5229]
20201230 [5226, 5229]
20201231 [5226, 5229]
20210104 [5226, 5229]
20210105 [5226, 5229, 5273, 5329, 5377, 5384, 5386]
20210106 [5226, 5229, 5273, 5326, 5377]
20210107 [5226, 5229, 5273]
20210108 [5226, 5229]
[5264 5281 5384 5225 5273 5269 5226 5230 5290 5222 5275 5287 5292 5276
 5332 5291 5289 5377 5229 5386 5232 5326 5329]
SSE
20201214 [537301]
20201215 [537301]
20201216 [537301]
20201217 [537301]
20201218 [537301]
20201221 [537301]
20201222 [537301]
20201223 [537301]
20201224 [537301]
20201225 [537301, 537501]
20201228 [527301, 537501]
20201229 [527301, 537501]
20201230 [527301, 537501]
20201231 [527301, 532801, 537501]
20210104 [527301, 532801, 537501]
20210105 [527301, 528101, 532801, 537401, 537501]
202101

In [9]:
orderLog[((orderLog['colo'].str[:5] == 'zs_52') & (orderLog['exchange'] == 'SZE')) | \
        ((orderLog['colo'].str[:5] == 'zt_52') & (orderLog['exchange'] == 'SSE'))].groupby(['colo', 'date'])['accCode'].unique().apply(lambda x: np.sort(x))

colo      date    
zs_52_06  20201214    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201215    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201216    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201217    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201218    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201221    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201222    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201223    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201224    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201225    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201228    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201229    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201230    [5222, 5232, 5269, 5273, 5275, 5287, 5332, 5384]
          20201231    [5222, 5232, 5269, 5273, 5275, 5287,